In [ ]:
# Five-minute runtime

import pandas as pd
import numpy as np

In [ ]:
claims = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/heritage health prize/heritage dataset/release3/Claims.csv')

members = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/heritage health prize/heritage dataset/release3/Members.csv')
drug_count = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/heritage health prize/heritage dataset/release3/DrugCount.csv')
lab_count = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/heritage health prize/heritage dataset/release3/LabCount.csv')

dih_y2 = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/heritage health prize/heritage dataset/release3/DaysInHospital_Y2.csv')
dih_y3 = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/heritage health prize/heritage dataset/release3/DaysInHospital_Y3.csv')
# target_df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/heritage health prize/heritage dataset/release3/Target.csv')

claims_1 = claims[claims['Year']=='Y1'].merge(dih_y2, how='left', on='MemberID')

# Pre-processing Data

In [ ]:
# convert text values to integer
# pay delay
def pay_delay_to_int(text_val):
  if text_val=='162+':
    return 162
  else:
    return int(text_val)

# days since first service (claim)
def dsfs_to_int(text_val):
  if text_val=='0- 1 month':
    return 1
  elif text_val=='1- 2 months':
    return 2
  elif text_val=='2- 3 months':
    return 3
  elif text_val=='3- 4 months':
    return 4
  elif text_val=='4- 5 months':
    return 5
  elif text_val=='5- 6 months':
    return 6
  elif text_val=='6- 7 months':
    return 7
  elif text_val=='7- 8 months':
    return 8
  elif text_val=='8- 9 months':
    return 9
  elif text_val=='9-10 months':
    return 10
  elif text_val=='10-11 months':
    return 11
  elif text_val=='11-12 months':
    return 12

# charson index
def charlson_ind_to_int(text_val):
  if text_val=='0':
    return 0
  elif text_val=='1-2':
    return 2
  elif text_val=='3-4':
    return 4
  elif text_val=='5+':
    return 6

# length of stay
def los_to_int(text_val):
  if text_val=='1 day':
    return 1
  elif text_val=='2 days':
    return 2
  elif text_val=='3 days':
    return 3
  elif text_val=='4 days':
    return 4
  elif text_val=='5 days':
    return 5
  elif text_val=='6 days':
    return 6
  elif text_val=='1- 2 weeks':
    return 11
  elif text_val=='2- 4 weeks':
    return 21
  elif text_val=='4- 8 weeks':
    return 42
  elif text_val=='26+ weeks':
    return 180

# drug count
def drug_count_to_int(text_val):
  if text_val=='7+':
    return 7
  else:
    return int(text_val)

# lab count
def lab_count_to_int(text_val):
  if text_val=='10+':
    return 10
  else:
    return int(text_val)

# make some custom functions
# get range (max - min) of a series
def get_range(series):
  return np.max(series) - np.min(series)

# calculate standard deviation of a series
def std_dev(series):
  if len(series)==1:
    return 0
  else:
    return series.std()

# get total number of unknown length of stay
def get_los_tot_unknown(row):
  if (pd.isna(row['LengthOfStay'])==True) & (row['SupLOS']==0):
    return 1
  else:
    return 0

# get total number of suppressed length of stay
def get_los_tot_suppressed(row):
  if (pd.isna(row['LengthOfStay'])==True) & (row['SupLOS']==1):
    return 1
  else:
    return 0

# get total number of known length of stay
def get_los_tot_known(row):
  if (pd.isna(row['LengthOfStay'])==False):
    return 1
  else:
    return 0

# function to see if patients will visit hospital
def count_hospital_appearance(series):
  if series>0:
    return 1
  else:
    return 0

In [ ]:
claims.loc[:, 'PayDelay'] = claims['PayDelay'].apply(pay_delay_to_int)
claims.loc[:, 'DSFS'] = claims['DSFS'].apply(dsfs_to_int)
claims.loc[:, 'CharlsonIndex'] = claims['CharlsonIndex'].apply(charlson_ind_to_int)
claims.loc[:, 'LengthOfStay'] = claims['LengthOfStay'].apply(los_to_int)

drug_count.loc[:, 'DrugCount'] = drug_count['DrugCount'].apply(drug_count_to_int)
drug_count.loc[:, 'DSFS'] = drug_count['DSFS'].apply(dsfs_to_int)

lab_count.loc[:, 'LabCount'] = lab_count['LabCount'].apply(lab_count_to_int)
lab_count.loc[:, 'DSFS'] = lab_count['DSFS'].apply(dsfs_to_int)

claims.loc[:, 'los_tot_unknown'] = claims.apply(lambda row: get_los_tot_unknown(row), axis=1)
claims.loc[:, 'los_tot_suppressed'] = claims.apply(lambda row: get_los_tot_suppressed(row), axis=1)
claims.loc[:, 'los_tot_known'] = claims.apply(lambda row: get_los_tot_known(row), axis=1)

claims_1.loc[:, 'in_hospital'] = claims_1['DaysInHospital'].apply(count_hospital_appearance)

In [ ]:
# PCP
pcp_calc = claims_1.groupby('PCP', as_index=False).agg({
    'MemberID': 'nunique',
    'in_hospital': 'sum',
    'DaysInHospital': 'count'
}).rename(columns={
    'MemberID': 'number of unique visits',
    'DaysInHospital': 'number of visits'
})

# calculate the probability that a patient associated with a PCP will visit hospital
pcp_calc.loc[:, 'pcp_prob'] = pcp_calc['in_hospital'] / pcp_calc['number of visits']

# Provider
provider_calc = claims_1.groupby('ProviderID', as_index=False).agg({
    'MemberID': 'nunique',
    'in_hospital': 'sum',
    'DaysInHospital': 'count'
}).rename(columns={
    'MemberID': 'number of unique visits',
    'DaysInHospital': 'number of visits'
})

# calculate the probability that a patient associated with a Provider will visit hospital
provider_calc.loc[:, 'provider_prob'] = provider_calc['in_hospital'] / provider_calc['number of visits']

# Vendor
vendor_calc = claims_1.groupby('Vendor', as_index=False).agg({
    'MemberID': 'nunique',
    'in_hospital': 'sum',
    'DaysInHospital': 'count'
}).rename(columns={
    'MemberID': 'number of unique visits',
    'DaysInHospital': 'number of visits'
})

# calculate the probability that a patient associated with a Vendor will visit hospital
vendor_calc.loc[:, 'vendor_prob'] = vendor_calc['in_hospital'] / vendor_calc['number of visits']

# get the PCPs (Vendors or Providers) that are associated with a "critical mass" of unique patients, being 100 for PCP, 200 for Vendor and 250 for Provider
pcp_calc = pcp_calc[pcp_calc['number of unique visits']>100]
vendor_calc = vendor_calc[vendor_calc['number of unique visits']>200]
provider_calc = provider_calc[provider_calc['number of unique visits']>250]

claims_1 = claims_1.merge(pcp_calc[['PCP', 'pcp_prob']], how='left', on='PCP')
claims_1 = claims_1.merge(vendor_calc[['Vendor', 'vendor_prob']], how='left', on='Vendor')
claims_1 = claims_1.merge(provider_calc[['ProviderID', 'provider_prob']], how='left', on='ProviderID')

member_prob = claims_1.groupby('MemberID', as_index=False)[['pcp_prob', 'vendor_prob', 'provider_prob']].max()

In [ ]:
no_claims = claims.groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('no_claims')
no_providers = claims.groupby(['Year', 'MemberID'])['ProviderID'].nunique().rename('no_providers')
no_vendors = claims.groupby(['Year', 'MemberID'])['Vendor'].nunique().rename('no_vendors')
no_pcp = claims.groupby(['Year', 'MemberID'])['PCP'].nunique().rename('no_pcp')
no_placesvc = claims.groupby(['Year', 'MemberID'])['PlaceSvc'].nunique().rename('no_placesvc')
no_specialty = claims.groupby(['Year', 'MemberID'])['Specialty'].nunique().rename('no_specialty')
no_pcg = claims.groupby(['Year', 'MemberID'])['PrimaryConditionGroup'].nunique().rename('no_pcg')
no_pg = claims.groupby(['Year', 'MemberID'])['ProcedureGroup'].nunique().rename('no_pg')

paydelay = claims.groupby(['Year', 'MemberID'])['PayDelay'].agg(['max', 'min', 'mean', std_dev]).rename(columns={
    'max': 'max_paydelay',
    'min': 'min_paydelay',
    'mean': 'avg_paydelay',
    'std_dev': 'std_paydelay'
})

los = claims.groupby(['Year', 'MemberID'])['LengthOfStay'].agg(['max', 'min', 'mean', std_dev]).rename(columns={
    'max': 'max_los',
    'min': 'min_los',
    'mean': 'avg_los',
    'std_dev': 'std_los'
})

los_tot_unknown = claims.groupby(['Year', 'MemberID'])['los_tot_unknown'].sum()
los_tot_suppressed = claims.groupby(['Year', 'MemberID'])['los_tot_suppressed'].sum()
los_tot_known = claims.groupby(['Year', 'MemberID'])['los_tot_known'].sum()

charlsonindex = claims.groupby(['Year', 'MemberID'])['CharlsonIndex'].agg(['max', 'min', 'mean', std_dev, get_range]).rename(columns={
    'max': 'max_charlsonindex',
    'min': 'min_charlsonindex',
    'mean': 'avg_charlsonindex',
    'std_dev': 'std_charlsonindex',
    'get_range': 'range_charlsonindex'
})

dsfs = claims.groupby(['Year', 'MemberID'])['DSFS'].agg(['max', 'min', 'mean', std_dev, get_range]).rename(columns={
    'max': 'max_dsfs',
    'min': 'min_dsfs',
    'mean': 'avg_dsfs',
    'std_dev': 'std_dsfs',
    'get_range': 'range_dsfs'
})

pcg0 = claims[claims['PrimaryConditionGroup']=='NEUMENT'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg0')
pcg1 = claims[claims['PrimaryConditionGroup']=='METAB3'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg1')
pcg2 = claims[claims['PrimaryConditionGroup']=='FXDISLC'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg2')
pcg3 = claims[claims['PrimaryConditionGroup']=='TRAUMA'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg3')
pcg4 = claims[claims['PrimaryConditionGroup']=='RESPR4'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg4')
pcg5 = claims[claims['PrimaryConditionGroup']=='INFEC4'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg5')
pcg6 = claims[claims['PrimaryConditionGroup']=='MISCHRT'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg6')
pcg7 = claims[claims['PrimaryConditionGroup']=='ARTHSPIN'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg7')
pcg8 = claims[claims['PrimaryConditionGroup']=='ROAMI'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg8')
pcg9 = claims[claims['PrimaryConditionGroup']=='HEART2'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg9')
pcg10 = claims[claims['PrimaryConditionGroup']=='MSC2a3'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg10')
pcg11 = claims[claims['PrimaryConditionGroup']=='PNEUM'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg11')
pcg12 = claims[claims['PrimaryConditionGroup']=='MISCL5'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg12')
pcg13 = claims[claims['PrimaryConditionGroup']=='GIOBSENT'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg13')
pcg14 = claims[claims['PrimaryConditionGroup']=='RENAL3'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg14')
pcg15 = claims[claims['PrimaryConditionGroup']=='GYNEC1'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg15')
pcg16 = claims[claims['PrimaryConditionGroup']=='AMI'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg16')
pcg17 = claims[claims['PrimaryConditionGroup']=='UTI'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg17')
pcg18 = claims[claims['PrimaryConditionGroup']=='COPD'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg18')
pcg19 = claims[claims['PrimaryConditionGroup']=='GIBLEED'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg19')
pcg20 = claims[claims['PrimaryConditionGroup']=='HIPFX'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg20')
pcg21 = claims[claims['PrimaryConditionGroup']=='SKNAUT'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg21')
pcg22 = claims[claims['PrimaryConditionGroup']=='CANCRB'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg22')
pcg23 = claims[claims['PrimaryConditionGroup']=='SEIZURE'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg23')
pcg24 = claims[claims['PrimaryConditionGroup']=='CHF'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg24')
pcg25 = claims[claims['PrimaryConditionGroup']=='PRGNCY'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg25')
pcg26 = claims[claims['PrimaryConditionGroup']=='METAB1'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg26')
pcg27 = claims[claims['PrimaryConditionGroup']=='HEART4'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg27')
pcg28 = claims[claims['PrimaryConditionGroup']=='ODaBNCA'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg28')
pcg29 = claims[claims['PrimaryConditionGroup']=='HEMTOL'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg29')
pcg30 = claims[claims['PrimaryConditionGroup']=='GYNECA'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg30')
pcg31 = claims[claims['PrimaryConditionGroup']=='APPCHOL'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg31')
pcg32 = claims[claims['PrimaryConditionGroup']=='MISCL1'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg32')
pcg33 = claims[claims['PrimaryConditionGroup']=='RENAL2'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg33')
pcg34 = claims[pd.isna(claims['PrimaryConditionGroup'])==True].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg34')
pcg35 = claims[claims['PrimaryConditionGroup']=='STROKE'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg35')
pcg36 = claims[claims['PrimaryConditionGroup']=='PERVALV'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg36')
pcg37 = claims[claims['PrimaryConditionGroup']=='CANCRA'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg37')
pcg38 = claims[claims['PrimaryConditionGroup']=='CATAST'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg38')
pcg39 = claims[claims['PrimaryConditionGroup']=='FLaELEC'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg39')
pcg40 = claims[claims['PrimaryConditionGroup']=='SEPSIS'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg40')
pcg41 = claims[claims['PrimaryConditionGroup']=='LIVERDZ'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg41')
pcg42 = claims[claims['PrimaryConditionGroup']=='PNCRDZ'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg42')
pcg43 = claims[claims['PrimaryConditionGroup']=='RENAL1'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg43')
pcg44 = claims[claims['PrimaryConditionGroup']=='CANCRM'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg44')
pcg45 = claims[claims['PrimaryConditionGroup']=='PERINTL'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pcg45')

sp0 = claims[claims['Specialty']=='Surgery'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp0')
sp1 = claims[claims['Specialty']=='Internal'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp1')
sp2 = claims[claims['Specialty']=='Laboratory'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp2')
sp3 = claims[claims['Specialty']=='Pediatrics'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp3')
sp4 = claims[claims['Specialty']=='Rehabilitation'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp4')
sp5 = claims[claims['Specialty']=='Diagnostic Imaging'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp5')
sp6 = claims[claims['Specialty']=='Anesthesiology'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp6')
sp7 = claims[claims['Specialty']=='Emergency'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp7')
sp8 = claims[claims['Specialty']=='General Practice'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp8')
sp9 = claims[claims['Specialty']=='Other'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp9')
sp10 = claims[claims['Specialty']=='Obstetrics and Gynecology'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp10')
sp11 = claims[claims['Specialty']=='Pathology'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp11')
sp12 = claims[pd.isna(claims['Specialty'])==True].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('sp12')

pg0 = claims[claims['ProcedureGroup']=='MED'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg0')
pg1 = claims[claims['ProcedureGroup']=='EM'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg1')
pg2 = claims[claims['ProcedureGroup']=='SCS'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg2')
pg3 = claims[claims['ProcedureGroup']=='RAD'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg3')
pg4 = claims[claims['ProcedureGroup']=='ANES'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg4')
pg5 = claims[claims['ProcedureGroup']=='SDS'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg5')
pg6 = claims[claims['ProcedureGroup']=='PL'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg6')
pg7 = claims[claims['ProcedureGroup']=='SIS'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg7')
pg8 = claims[claims['ProcedureGroup']=='SMS'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg8')
pg9 = claims[claims['ProcedureGroup']=='SGS'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg9')
pg10 = claims[claims['ProcedureGroup']=='SEOA'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg10')
pg11 = claims[claims['ProcedureGroup']=='SNS'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg11')
pg12 = claims[pd.isna(claims['ProcedureGroup'])==True].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg12')
pg13 = claims[claims['ProcedureGroup']=='SAS'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg13')
pg14 = claims[claims['ProcedureGroup']=='SRS'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg14')
pg15 = claims[claims['ProcedureGroup']=='SUS'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg15')
pg16 = claims[claims['ProcedureGroup']=='SO'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg16')
pg17 = claims[claims['ProcedureGroup']=='SMCD'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('pg17')

ps0 = claims[claims['PlaceSvc']=='Office'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('ps0')
ps1 = claims[claims['PlaceSvc']=='Independent Lab'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('ps1')
ps2 = claims[claims['PlaceSvc']=='Outpatient Hospital'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('ps2')
ps3 = claims[claims['PlaceSvc']=='Inpatient Hospital'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('ps3')
ps4 = claims[claims['PlaceSvc']=='Urgent Care'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('ps4')
ps5 = claims[claims['PlaceSvc']=='Ambulance'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('ps5')
ps6 = claims[claims['PlaceSvc']=='Home'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('ps6')
ps7 = claims[pd.isna(claims['PlaceSvc'])==True].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('ps7')
ps8 = claims[claims['PlaceSvc']=='Other'].groupby(['Year', 'MemberID']).count().iloc[:, 0].rename('ps8')

drugcount_sum = drug_count.groupby(['Year', 'MemberID'])['DrugCount'].agg(['min', 'max', 'mean', 'count']).rename(columns={
    'min': 'min_drugcount',
    'max': 'max_drugcount',
    'mean': 'avg_drugcount',
    'count': 'count_drugcount'
})

labcount_sum = lab_count.groupby(['Year', 'MemberID'])['LabCount'].agg(['min', 'max', 'mean', 'count']).rename(columns={
    'min': 'min_labcount',
    'max': 'max_labcount',
    'mean': 'avg_labcount',
    'count': 'count_labcount'
})

In [ ]:
# handling null values
los = los.fillna({'max_los': 0, 'min_los': 0, 'avg_los': 0, 'std_los': -1})

dsfs = dsfs.fillna({'max_dsfs': 0, 'min_dsfs': 0, 'avg_dsfs': 0, 'std_dsfs': -1, 'range_dsfs': -1})

charlsonindex = charlsonindex.fillna({'max_charlsonindex': 0, 'min_charlsonindex': 0, 'avg_charlsonindex': 0, 'std_charlsonindex': -1, 'range_charlsonindex': -1})

In [ ]:
# concatenate all member-level data from previous calculations
claims_per_member = pd.concat([
                                no_claims, no_providers, no_vendors, no_pcp, no_placesvc, no_specialty, no_pcg,
                                no_pg, paydelay, los, los_tot_unknown, los_tot_suppressed, los_tot_known, charlsonindex, dsfs,
                                pcg0, pcg1, pcg2, pcg3, pcg4, pcg5, pcg6, pcg7, pcg8, pcg9, pcg10, pcg11, pcg12,
                                pcg13, pcg14, pcg15, pcg16, pcg17, pcg18, pcg19, pcg20, pcg21, pcg22, pcg23, pcg24,
                                pcg25, pcg26, pcg27, pcg28, pcg29, pcg30, pcg31, pcg32, pcg33, pcg34, pcg35, pcg36,
                                pcg37, pcg38, pcg39, pcg40, pcg41, pcg42, pcg43, pcg44, pcg45,
                                sp0, sp1, sp2, sp3, sp4, sp5, sp6, sp7, sp8, sp9, sp10, sp11, sp12,
                                pg0, pg1, pg2, pg3, pg4, pg5, pg6, pg7, pg8, pg9, pg10, pg11, pg12, pg13, pg14,
                                pg15, pg16, pg17,
                                ps0, ps1, ps2, ps3, ps4, ps5, ps6, ps7, ps8
                        ], axis=1).reset_index()

# create dummy variables for age and sex features from members dataframe
members = pd.get_dummies(data=members, columns=['AgeAtFirstClaim', 'Sex'], prefix=['age', 'sex'], dummy_na=True)

# prepare trainset and test set
dih_y2.loc[:, 'Year'] = 'Y1'
dih_y2.loc[:, 'trainset'] = 1

dih_y3.loc[:, 'Year'] = 'Y2'
dih_y3.loc[:, 'trainset'] = 0
dih = pd.concat([dih_y2, dih_y3])

# dih (days in hospital) merges with members
df1 = dih.merge(members, how='left', on='MemberID')

# df1 merges with claims_per_member
df2 = df1.merge(claims_per_member, how='left', on=['Year', 'MemberID'])

# df2 merges with drugcount_sum
df3 = df2.merge(drugcount_sum, how='left', on=['Year', 'MemberID'])

# df3 merges with labcount_sum
df4 = df3.merge(labcount_sum, how='left', on=['Year', 'MemberID'])

# df4 merges with member_prob
# probabilities of members in year 1 are also applied to members in year 2 because year 3 DaysInHospital data are not available in year 2
df5 = df4.merge(member_prob, how='left', on='MemberID')

In [ ]:
# handling missing values from drugcount and labcount
def count_nan_drugcount(series):
  if pd.isna(series)==True:
    return 1
  else:
    return 0

def count_nan_labcount(series):
  if pd.isna(series)==True:
    return 1
  else:
    return 0

In [ ]:
df5.loc[:, 'drugcount_nan'] = df5['max_drugcount'].apply(count_nan_drugcount)
df5.loc[:, 'labcount_nan'] = df5['max_labcount'].apply(count_nan_labcount)

In [ ]:
# replace NaN values in df5 with 0
df5 = df5.fillna(0)

In [ ]:
df5.head()

,MemberID,ClaimsTruncated,DaysInHospital,Year,trainset,age_0-9,age_10-19,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,age_80+,age_nan,sex_F,sex_M,sex_nan,no_claims,no_providers,no_vendors,no_pcp,no_placesvc,no_specialty,no_pcg,no_pg,max_paydelay,min_paydelay,avg_paydelay,std_paydelay,max_los,min_los,avg_los,std_los,los_tot_unknown,los_tot_suppressed,los_tot_known,max_charlsonindex,min_charlsonindex,avg_charlsonindex,...,pg0,pg1,pg2,pg3,pg4,pg5,pg6,pg7,pg8,pg9,pg10,pg11,pg12,pg13,pg14,pg15,pg16,pg17,ps0,ps1,ps2,ps3,ps4,ps5,ps6,ps7,ps8,min_drugcount,max_drugcount,avg_drugcount,count_drugcount,min_labcount,max_labcount,avg_labcount,count_labcount,pcp_prob,vendor_prob,provider_prob,drugcount_nan,labcount_nan
0,24027423,0,0,Y1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,5,3,3,1,2,3,3,4,35,27,30.400000,4.219005,0.0,0.0,0.0,-1.0,5,0,0,4,0,2.8,...,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.000000,1.0,7.0,7.0,7.000000,1.0,0.257737,0.299153,0.239850,0,0
1,98324177,0,0,Y1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,6,3,3,1,2,2,2,4,146,23,70.666667,59.308234,0.0,0.0,0.0,-1.0,6,0,0,0,0,0.0,...,0.0,2.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.000000,1.0,7.0,7.0,7.000000,1.0,0.265180,0.248000,0.242471,0,0
2,33899367,1,1,Y1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,41,8,8,1,3,6,8,6,162,13,45.000000,31.216983,0.0,0.0,0.0,-1.0,41,0,0,0,0,0.0,...,5.0,22.0,1.0,3.0,0.0,0.0,1.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,6.0,2.300000,10.0,1.0,1.0,1.000000,1.0,0.106716,0.327027,0.209908,0,0
3,5481382,0,1,Y1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,12,6,5,1,3,5,5,4,162,28,68.000000,48.151277,0.0,0.0,0.0,-1.0,12,0,0,0,0,0.0,...,0.0,5.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,1.111111,9.0,1.0,2.0,1.666667,3.0,0.213570,0.245260,0.239850,0,0
4,69908334,0,0,Y1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,17,9,6,1,3,5,5,6,77,22,48.117647,16.773798,0.0,0.0,0.0,-1.0,17,0,0,0,0,0.0,...,4.0,6.0,1.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,1.777778,9.0,8.0,8.0,8.000000,1.0,0.268316,0.245260,0.253925,0,0


In [ ]:
# df5.to_csv(r'/content/drive/MyDrive/Colab Notebooks/heritage health prize/hhp_final_data.csv', index=False)